In [1]:
from typing import (
    Optional,
    Tuple,
    List,
    Dict,
)
from numbers import Number
import sys
import os
import pathlib
import getopt
import logging
import re
import numpy as np


(N, M) = 5, 4
BASE = None
END = None

DIRECTIONS = None
DIRECTION_TO_MOVE = None
MOVE_TO_DIRECTION = None
MOVES = None
MOVE = None

Logger = None


def initialize(n, m) -> None:
    global N
    global M
    global BASE
    global END
    global DIRECTIONS 
    global DIRECTION_TO_MOVE
    global MOVE_TO_DIRECTION
    global MOVES
    global Move
    global Logger
    
    logging.basicConfig()
    logging.getLogger().setLevel(logging.DEBUG)
    Logger = logging.getLogger(__name__)

    N = n   # x / east direction
    M = m   # y / north direction
    
    BASE = np.zeros(2).astype(int)
    END = np.array([N-1,M-1])

    DIRECTION_TO_MOVE = {
        "NORTH": [0, 1], "EAST": [1, 0], "WEST": [-1, 0], "SOUTH": [0, -1]   
    }
    DIRECTIONS = list(DIRECTION_TO_MOVE.keys())

    MOVE_TO_DIRECTION = {
        tuple(DIRECTION_TO_MOVE["NORTH"]): "NORTH", 
        tuple(DIRECTION_TO_MOVE["EAST"]): "EAST", 
        tuple(DIRECTION_TO_MOVE["WEST"]): "WEST", 
        tuple(DIRECTION_TO_MOVE["SOUTH"]): "SOUTH", 
    }
    MOVES = [list(move) for move in MOVE_TO_DIRECTION.keys()]

    Position = np.array([0,0])
    Direction = DIRECTIONS[0]
    Move = MOVES[0]

In [2]:
def place(_position=None, _direction=None) -> (List, str, List):
    """Set the current position, direction, and move
    Returns: (Position, Direction, Move)
    """
    Logger.debug("place: position {}, direction {}".format(_position, _direction))
    
    global Position
    global Direction
    global Move

    _x, _y = (_position[0], _position[1]) if _position is not None else (None, None)
    _direction = _direction.upper() if _direction is not None else "NOWHERE"
    if isinstance(_x, Number) and isinstance(_y, Number) and _direction in DIRECTIONS:
        Position = [_x, _y]
        Direction = _direction
        Move = DIRECTION_TO_MOVE[Direction]

    Logger.debug("place: new positon {} direction {} move {}".format(
        Position, Direction, Move
    ))
    return Position, Direction, Move


def report() -> None:
    """Report the current location and direction"""
    template = "X: {} Y: {} Direction: {}"
    position, direction, _ = place()
    print(template.format(position[0], position[1], direction))

In [3]:
def is_inside(position) -> bool:
    """Check if the position is inside"""
    decision = np.all(position >= BASE) and np.all(position <= END)
    Logger.debug("position is {} and is inside is {}".format(
        position, decision
    ))
    return decision


def is_same_array(a, b) -> bool:
    return np.all(np.array(a) == np.array(b))

at_same_location = is_same_array
    
def move() -> List:
    """Move a step in the current direction if the destination is within the limit
    Returns new position
    """
    global Position
    
    moved = np.add(Position, Move)
    if is_inside(moved):
            Position = moved

    return Position

In [4]:
def _rotate(vector, theta):
    """Rotate the vector with theta degree clock-wise
    Args:
        vector: vector to rotate
        theta: degrees to rotate
    Return: 
        rotated vector
    """
    radian = np.radians(theta)
    rotation = np.array([
        [  np.cos(radian), np.sin(radian) ],
        [ -np.sin(radian), np.cos(radian) ]
    ])
    rotated = rotation.dot(vector).astype(int)
    Logger.debug("Current vector is {} rotation is {}. New vector is {}".format(
        vector, theta, rotated
    ))
    return rotated

def rotate(theta):
    """Rotate the current move vector with theta degree
    Args:
        theta: rotation degrees
    Returns: new move vector
    """
    global Move
    Move = _rotate(Move, theta)
    
    return Move

def left():
    return rotate(-90)
    
def right():
    return rotate(90)

In [5]:
COMMANDS = {
    "PLACE": place,
    "LEFT": left,
    "RIGHT": right,
    "MOVE": move,
    "REPORT": report
}

In [6]:
initialize(n=5, m=4)

assert MOVE_TO_DIRECTION[tuple([0,1])] == "NORTH"
assert at_same_location(left(), DIRECTION_TO_MOVE["WEST"])
assert at_same_location(left(), DIRECTION_TO_MOVE["SOUTH"])
assert at_same_location(left(), DIRECTION_TO_MOVE["EAST"])
assert at_same_location(left(), DIRECTION_TO_MOVE["NORTH"])

assert at_same_location(right(), DIRECTION_TO_MOVE["EAST"])
assert at_same_location(right(), DIRECTION_TO_MOVE["SOUTH"])
assert at_same_location(right(), DIRECTION_TO_MOVE["WEST"])
assert at_same_location(right(), DIRECTION_TO_MOVE["NORTH"])

DEBUG:__main__:Current vector is [0, 1] rotation is -90. New vector is [-1  0]
DEBUG:__main__:Current vector is [-1  0] rotation is -90. New vector is [ 0 -1]
DEBUG:__main__:Current vector is [ 0 -1] rotation is -90. New vector is [1 0]
DEBUG:__main__:Current vector is [1 0] rotation is -90. New vector is [0 1]
DEBUG:__main__:Current vector is [0 1] rotation is 90. New vector is [1 0]
DEBUG:__main__:Current vector is [1 0] rotation is 90. New vector is [ 0 -1]
DEBUG:__main__:Current vector is [ 0 -1] rotation is 90. New vector is [-1  0]
DEBUG:__main__:Current vector is [-1  0] rotation is 90. New vector is [0 1]


In [7]:
x, y, d = place([0, 0], "north")
destination = move()
assert at_same_location(destination, [0,1])

for i in range(M): # M is y/north direction
    move()
    report()

DEBUG:__main__:place: position [0, 0], direction north
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [0, 1]
DEBUG:__main__:position is [0 1] and is inside is True
DEBUG:__main__:position is [0 2] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0 2] direction NORTH move [0, 1]
DEBUG:__main__:position is [0 3] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0 3] direction NORTH move [0, 1]
DEBUG:__main__:position is [0 4] and is inside is False
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0 3] direction NORTH move [0, 1]
DEBUG:__main__:position is [0 4] and is inside is False
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0 3] direction NORTH move [0, 1]


X: 0 Y: 2 Direction: NORTH
X: 0 Y: 3 Direction: NORTH
X: 0 Y: 3 Direction: NORTH
X: 0 Y: 3 Direction: NORTH


In [8]:
x, y, d = place([0, 0], "east")
destination = move()
print(destination)
assert at_same_location(destination, [1,0])
report()

for i in range(N):
    move()
    report()

DEBUG:__main__:place: position [0, 0], direction east
DEBUG:__main__:place: new positon [0, 0] direction EAST move [1, 0]
DEBUG:__main__:position is [1 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [1 0] direction EAST move [1, 0]
DEBUG:__main__:position is [2 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [2 0] direction EAST move [1, 0]
DEBUG:__main__:position is [3 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [3 0] direction EAST move [1, 0]
DEBUG:__main__:position is [4 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [4 0] direction EAST move [1, 0]
DEBUG:__main__:position is [5 0] and is inside is False
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [4 0] direction EAST move [1, 0]
DEBUG:_

[1 0]
X: 1 Y: 0 Direction: EAST
X: 2 Y: 0 Direction: EAST
X: 3 Y: 0 Direction: EAST
X: 4 Y: 0 Direction: EAST
X: 4 Y: 0 Direction: EAST
X: 4 Y: 0 Direction: EAST


In [9]:
x, y, d = place([0, 0], "north")
new_move = COMMANDS["LEFT"]()
print(new_move)
assert is_same_array(new_move, DIRECTION_TO_MOVE["WEST"])
report()

for i in range(N):
    move()
    report()

DEBUG:__main__:place: position [0, 0], direction north
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [0, 1]
DEBUG:__main__:Current vector is [0, 1] rotation is -90. New vector is [-1  0]
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [-1  0]
DEBUG:__main__:position is [-1  0] and is inside is False
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [-1  0]
DEBUG:__main__:position is [-1  0] and is inside is False
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [-1  0]
DEBUG:__main__:position is [-1  0] and is inside is False
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [-1  0]
DEBUG:__main__:position is [-1  0] and is inside is False
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new

[-1  0]
X: 0 Y: 0 Direction: NORTH
X: 0 Y: 0 Direction: NORTH
X: 0 Y: 0 Direction: NORTH
X: 0 Y: 0 Direction: NORTH
X: 0 Y: 0 Direction: NORTH
X: 0 Y: 0 Direction: NORTH


In [10]:
x, y, d = place([0, 0], "north")
new_move = COMMANDS["RIGHT"]()
print(new_move)
assert is_same_array(new_move, DIRECTION_TO_MOVE["EAST"])
report()

for i in range(N):
    move()
    report()

DEBUG:__main__:place: position [0, 0], direction north
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [0, 1]
DEBUG:__main__:Current vector is [0, 1] rotation is 90. New vector is [1 0]
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [0, 0] direction NORTH move [1 0]
DEBUG:__main__:position is [1 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [1 0] direction NORTH move [1 0]
DEBUG:__main__:position is [2 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [2 0] direction NORTH move [1 0]
DEBUG:__main__:position is [3 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [3 0] direction NORTH move [1 0]
DEBUG:__main__:position is [4 0] and is inside is True
DEBUG:__main__:place: position None, direction None
DEBUG:__main__:place: new positon [4 0] direction N

[1 0]
X: 0 Y: 0 Direction: NORTH
X: 1 Y: 0 Direction: NORTH
X: 2 Y: 0 Direction: NORTH
X: 3 Y: 0 Direction: NORTH
X: 4 Y: 0 Direction: NORTH
X: 4 Y: 0 Direction: NORTH


In [11]:
def execute_command(line) -> Optional[str]:
    """Parse command line text and run corresponding command
    Args:
        line: command line
    Return:
        command executed e.g. PLACE or None if no execution
    """
    Logger.debug("execute_command: line [{}]".format(line))
    for command in COMMANDS.keys():
        if command == "PLACE":
            pattern = r'[\t\s]*^PLACE[\t\s]+([0-9]+)[\t\s]+([0-9]+)[\t\s]+(NORTH|EAST|WEST|SOUTH)'
            if match := re.search(pattern, line, re.IGNORECASE):
                x = int(match.group(1))
                y = int(match.group(2))
                direction = match.group(3).upper()

                Logger.debug("execute_command: matched command {}".format(
                    match.group(0).upper()
                ))
                COMMANDS[command]([x, y], direction)
        else:
            pattern = r'^[\t\s]*({})[\t\s]*'.format(command)
            if (match := re.search(pattern, line, re.IGNORECASE)):
                Logger.debug("execute_command: matched command {}".format(
                    match.group(0).upper()
                ))
                COMMANDS[command]()
            

    Logger.debug("execute_command: none executed.")
    return None

In [12]:
def read_lines(path: str) -> str:
    Logger.debug("read_lines: path [{}]".format(path))
    _file = pathlib.Path(path)
    if not _file.is_file():
        raise ValueError("file {} does not exist or non file".format(path))
    else:
        with _file.open() as f:
            for line in f:
                yield line

                
def process_commands(lines) -> None:
    while True:
        try:
            execute_command(next(lines))

        except StopIteration:
            break

In [14]:
def get_path(argv) -> Optional[str]:
    Logger.debug("get_path: argv [{}]".format(argv))
    path: str = None
    try:
        opts, args = getopt.getopt(argv[1:], "hf:")
        Logger.debug("get_path opts {} args {}".format(
            opts, args
        ))
    except getopt.GetoptError:
        Logger.error("Invalid command line")
        print("{} -f <path>".format(
            sys.argv[0]
        ))
        return None

    for opt, arg in opts:
        if opt == '-h':
            print("{} -f <path>".format(
                sys.argv[0]
            ))
            return None
        elif opt in ("-f") :
            Logger.debug('command file is {}'.format(arg))
            return arg

In [16]:
def main(argv):
    if path := get_path(argv[1:]):
        process_commands(read_lines(path))

DEBUG:__main__:get_command_file_path: argv [('-f', '/home/oonisim/home/repository/git/oonisim/python-programs/Python/generator/hoge')]
DEBUG:__main__:get_command_file_path opts [] args ('/home/oonisim/home/repository/git/oonisim/python-programs/Python/generator/hoge',)


In [18]:
if __name__ == "__main__":
    main(sys.argv)
else:
    argv = ("jupyter", '-f', '/home/oonisim/home/repository/git/oonisim/python-programs/Python/generator/hoge')
    main(argv)

DEBUG:__main__:get_command_file_path: argv [['-f', '/home/oonisim/.local/share/jupyter/runtime/kernel-e3197282-e777-4deb-ae63-b9f0ae2f6068.json']]
DEBUG:__main__:get_command_file_path opts [] args ['/home/oonisim/.local/share/jupyter/runtime/kernel-e3197282-e777-4deb-ae63-b9f0ae2f6068.json']
